# 007_DSME_BTS_R
## DSME BTS_운반처리 데이터

## 데이터 처리 과정
### 1. 시간 연산을 위한 형변환
### 2. 원래 하려던 ifelse가 POSIXlt 형식에서 안먹히는 것 같음
### 3. st, et에 따라 row split & 시간 연산
### 4. split해서 처리 후 rbind & 시간 값 table 확인
### 5. 시간 값 작업
### 6. JOIN
### 7. 특정 Column명 바꾸기

In [1]:
library(tidyverse)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
√ ggplot2 2.2.1     √ purrr   0.2.4
√ tibble  1.4.1     √ dplyr   0.7.4
√ tidyr   0.7.2     √ stringr 1.2.0
√ readr   1.1.1     √ forcats 0.2.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
bts <- read.csv("C:\\IT\\project\\dsme\\BTS 운반실적_수정.csv")
head(bts)

MOVE_ACTL_ID,PCS_LEFT,PCS_TOP,ROT_ANGL,ACTL_RGSR_EQP_CODE,ACTL_INDC,RMRK,MOVE_OBJT_ID,MOVE_REQ_ID,RGSR_USER_ID,...,ARUD_ADDR_NO,ALRM_INDC,BLK_FLAG,CMPL_INDC,MNT_AREA_NO,MNT_BLK_FLAG,JIG_CODE,JIG_QTY,MNT_PCS_LEFT,MNT_PCS_TOP
20180111_333,173017.6,252124.5,NA,TP,,,5441_123,20180111_333,,...,NA,NA,,NA,GB1S03,NA,,NA,0,0
20180112_287,172548.0,252400.4,0,TP,,도경(반출),2488_865,20180112_287,Migration,...,NA,NA,,NA,NP3S,NA,,0,0,0
20180111_384,172933.5,252196.8,NA,TP,,,2431_223,20180111_384,,...,NA,NA,,NA,GB2W04,NA,,NA,0,0
20180111_403,172972.3,252278.5,NA,TP,,,2430_823,20180111_403,,...,NA,NA,,NA,GB2W03,NA,,NA,0,0
20180111_0197,172423.6,252651.4,NA,TP,,,2430_193,20180111_0197,,...,NA,NA,,NA,3D1W03,NA,,NA,0,0
20180105_0093,172232.7,253279.7,0,TP,,,2443_221,20180105_0093,Migration,...,NA,NA,,NA,1DW4,NA,,0,0,0


In [3]:
btime <- select(bts, MOVE_ACTL_ID, ACTL_MOVE_TIME_FROM, ACTL_MOVE_TIME_TO, ACTL_MOVE_DATE_FROM, ACTL_MOVE_DATE_TO)
colnames(btime) <- c('key', 'starttime', 'endtime', 'startdate', 'enddate')
head(btime)

key,starttime,endtime,startdate,enddate
20180111_333,2038,2043,2018-01-11,2018-01-11
20180112_287,2304,2310,2018-01-12,2018-01-12
20180111_384,2125,2128,2018-01-11,2018-01-11
20180111_403,2114,2124,2018-01-11,2018-01-11
20180111_0197,2039,2041,2018-01-11,2018-01-11
20180105_0093,1035,1056,2018-01-05,2018-01-05


## 1. 시간 연산을 위한 형변환

In [4]:
btime$startdate <- as.POSIXlt(btime$startdate)
btime$enddate <- as.POSIXlt(btime$enddate)
bt2 <- btime
str(bt2)

'data.frame':	8854 obs. of  5 variables:
 $ key      : Factor w/ 8854 levels "20180102_0001",..: 4501 5005 4543 4559 4184 1404 4191 4320 4390 4442 ...
 $ starttime: int  2038 2304 2125 2114 2039 1035 2044 1114 842 2033 ...
 $ endtime  : int  2043 2310 2128 2124 2041 1056 2051 1121 844 2038 ...
 $ startdate: POSIXlt, format: "2018-01-11" "2018-01-12" ...
 $ enddate  : POSIXlt, format: "2018-01-11" "2018-01-12" ...


## 2. 원래 하려던 ifelse가 POSIXlt 형식에서 안먹히는 것 같음

In [ ]:
# 왜 안되는지 모르겠음
'''
bt2$startdate <- ifelse(bt2$starttime > 2400, bt2$startdate + (24*3600), bt2$startdate)
bt2$enddate <- ifelse(bt2$endttime > 2400, bt2$enddate + (24*3600), bt2$enddate)
bt2$starttime <- ifelse(bt2$strattime > 2400, bt2$starttime - 2400, bt2$starttime)
bt2$endtime <- ifelse(bt2$endtime > 2400, bt2$endtime - 2400, bt2$endtime)

bt2$startdate <- ifelse(bt2$starttime > 2400, bt2$startdate, bt2$enddate)
head(bt2)
'''

## 3. st, et에 따라 row split & 시간 연산

In [5]:
# 조건에 따라 잘라서 연산하기로함
bt3 <- bt2[bt2$starttime > 2400 & bt2$endtime > 2400, ]
bt4 <- bt2[bt2$starttime > 2400 & bt2$endtime <= 2400, ]
bt5 <- bt2[bt2$starttime <= 2400 & bt2$endtime > 2400, ]
bt6 <- bt2[bt2$starttime <= 2400 & bt2$endtime <= 2400, ]

bt3$startdate <- bt3$startdate + (24*3600)
bt3$enddate <- bt3$enddate + (24*3600)
bt3$starttime <- bt3$starttime - 2400
bt3$endtime <- bt3$endtime - 2400

bt4$startdate <- bt4$startdate + (24*3600)
bt4$starttime <- bt4$starttime - 2400

bt5$enddate <- bt5$enddate + (24*3600)
bt5$endtime <- bt5$endtime - 2400

## 4. split해서 처리 후 rbind & 시간 값 table 확인

In [6]:
bt7 <- rbind(bt3, bt4, bt5, bt6)
head(bt6)
table(nchar(bt7$starttime))
table(nchar(bt7$endtime))

key,starttime,endtime,startdate,enddate
20180111_333,2038,2043,2018-01-11,2018-01-11
20180112_287,2304,2310,2018-01-12,2018-01-12
20180111_384,2125,2128,2018-01-11,2018-01-11
20180111_403,2114,2124,2018-01-11,2018-01-11
20180111_0197,2039,2041,2018-01-11,2018-01-11
20180105_0093,1035,1056,2018-01-05,2018-01-05



   1    2    3    4 
  10    6 2420 6418 


   1    2    3    4 
  14    7 2379 6454 

## 5. 시간 값 작업

In [7]:
# 여기서 시간 작업을 하고 key로 join 할 예정
bt7$starttime <- as.character(bt7$starttime)
bt7$endtime <- as.character(bt7$endtime)

bt7$starttime <- ifelse(nchar(bt7$starttime) == 1, paste0('000', bt7$starttime), bt7$starttime)
bt7$starttime <- ifelse(nchar(bt7$starttime) == 2, paste0('00', bt7$starttime), bt7$starttime)
bt7$starttime <- ifelse(nchar(bt7$starttime) == 3, paste0('0', bt7$starttime), bt7$starttime)
bt7$endtime <- ifelse(nchar(bt7$endtime) == 1, paste0('000', bt7$endtime), bt7$endtime)
bt7$endtime <- ifelse(nchar(bt7$endtime) == 2, paste0('00', bt7$endtime), bt7$endtime)
bt7$endtime <- ifelse(nchar(bt7$endtime) == 3, paste0('0', bt7$endtime), bt7$endtime)

table(nchar(bt7$starttime))
table(nchar(bt7$endtime))


   4 
8854 


   4 
8854 

In [8]:
head(bt7)

,key,starttime,endtime,startdate,enddate
28,20180111_0294,0121,0141,2018-01-12,2018-01-12
30,20180111_0303,0206,0222,2018-01-12,2018-01-12
46,20180109_0373,0517,0559,2018-01-10,2018-01-10
54,20180109_0374,0555,0601,2018-01-10,2018-01-10
59,20180109_141,0141,0148,2018-01-10,2018-01-10
63,20180109_214,0442,0442,2018-01-10,2018-01-10


In [9]:
bt8 <- bt7
bt8$starttime <- paste0(substr(bt8$starttime, 1, 2), ":", substr(bt8$starttime, 3, 4))
bt8$endtime <- paste0(substr(bt8$endtime, 1, 2), ":", substr(bt8$endtime, 3, 4))

bt8$startdate <- as.character(bt8$startdate)
bt8$enddate <- as.character(bt8$enddate)

bt8$sdt <- paste(bt8$startdate, bt8$starttime)
bt8$edt <- paste(bt8$enddate, bt8$endtime)

bt8$sdt <- paste0(bt8$sdt, ':00')
bt8$edt <- paste0(bt8$edt, ':00')

head(bt8)

,key,starttime,endtime,startdate,enddate,sdt,edt
28,20180111_0294,01:21,01:41,2018-01-12,2018-01-12,2018-01-12 01:21:00,2018-01-12 01:41:00
30,20180111_0303,02:06,02:22,2018-01-12,2018-01-12,2018-01-12 02:06:00,2018-01-12 02:22:00
46,20180109_0373,05:17,05:59,2018-01-10,2018-01-10,2018-01-10 05:17:00,2018-01-10 05:59:00
54,20180109_0374,05:55,06:01,2018-01-10,2018-01-10,2018-01-10 05:55:00,2018-01-10 06:01:00
59,20180109_141,01:41,01:48,2018-01-10,2018-01-10,2018-01-10 01:41:00,2018-01-10 01:48:00
63,20180109_214,04:42,04:42,2018-01-10,2018-01-10,2018-01-10 04:42:00,2018-01-10 04:42:00


## 6. JOIN

In [10]:
btstime <- select(bt8, key, sdt, edt)
head(btstime)

bts_mod <- left_join(bts, btstime, by = c("MOVE_ACTL_ID" = "key"))
str(bts_mod)

,key,sdt,edt
28,20180111_0294,2018-01-12 01:21:00,2018-01-12 01:41:00
30,20180111_0303,2018-01-12 02:06:00,2018-01-12 02:22:00
46,20180109_0373,2018-01-10 05:17:00,2018-01-10 05:59:00
54,20180109_0374,2018-01-10 05:55:00,2018-01-10 06:01:00
59,20180109_141,2018-01-10 01:41:00,2018-01-10 01:48:00
63,20180109_214,2018-01-10 04:42:00,2018-01-10 04:42:00


'data.frame':	8854 obs. of  48 variables:
 $ MOVE_ACTL_ID       : Factor w/ 8854 levels "20180102_0001",..: 4501 5005 4543 4559 4184 1404 4191 4320 4390 4442 ...
 $ PCS_LEFT           : num  173018 172548 172934 172972 172424 ...
 $ PCS_TOP            : num  252125 252400 252197 252279 252651 ...
 $ ROT_ANGL           : int  NA 0 NA NA NA 0 NA NA NA NA ...
 $ ACTL_RGSR_EQP_CODE : Factor w/ 2 levels "PC","TP": 2 2 2 2 2 2 2 2 2 2 ...
 $ ACTL_INDC          : Factor w/ 3 levels "","N","Y": 1 1 1 1 1 1 1 1 1 1 ...
 $ RMRK               : Factor w/ 40 levels "","1000톤으로작업",..: 1 17 1 1 1 1 1 1 1 1 ...
 $ MOVE_OBJT_ID       : Factor w/ 1376 levels "1448_122","2230_227",..: 1026 834 289 250 187 443 187 261 1056 1063 ...
 $ MOVE_REQ_ID        : Factor w/ 8851 levels "20180102_0001",..: 4501 5005 4543 4559 4184 1404 4191 4320 4390 4442 ...
 $ RGSR_USER_ID       : Factor w/ 3 levels "","Migration",..: 1 2 1 1 1 2 1 1 1 1 ...
 $ MNT_USER_ID        : Factor w/ 2 levels "","tester": 1 1 1 1 1 1 1 1

## 7. 특정 Column명 바꾸기

In [11]:
colnames(bts_mod)[which(names(bts_mod) == "sdt")] <- "START_TIME"
colnames(bts_mod)[which(names(bts_mod) == "edt")] <- "END_TIME"

In [ ]:
write.csv(bts_mod, "BTS_운반실적_TIME.csv", row.names = FALSE)

## 끝